<a href="https://colab.research.google.com/github/DamodaraBarbosa/estudos_web_scraping/blob/main/Scraping_Xbox_Series_Metacritic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importando as bibliotecas que serão utilizadas


Pandas, urllib e bs4

In [1]:
import pandas as pd
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup

## Definindo algumas funções

Função para obter o metascore do jogo

In [2]:
# função para obtenção da nota do metascore

def metascore_score(mscore):
  if 'positive' in str(mscore):
    meta = mscore.find('div', {'class': 'metascore_w large game positive'}).get_text()
  elif 'mixed' in str(mscore):
    meta = mscore.find('div', {'class': 'metascore_w large game mixed'}).get_text()
  else:
    meta = mscore.find('div', {'class': 'metascore_w large game negative'}).get_text()
  
  return meta

Função para obter o userscore do jogo

In [3]:
# função para obtenção da nota do userscore

def userscore_score(uscore): 
  if 'positive' in str(uscore):
    user = uscore.find('div', {'class': 'metascore_w user large game positive'}).get_text()
  elif 'mixed' in str(uscore):
    user = uscore.find('div', {'class': 'metascore_w user large game mixed'}).get_text()
  elif 'negative' in str(uscore):
    user = uscore.find('div', {'class': 'metascore_w user large game negative'}).get_text()
  else:
    user = uscore.find('div', {'class': 'metascore_w user large game tbd'}).get_text()
  
  return user

## Obtenção dos dados

Criando rotina de scraping

In [4]:
# o dado de cada jogo será armazenado no dicionário 'game' e cada 'game' na lista 'games'

game = dict()
games = list()

In [5]:
for i in range(0, 3):
  headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

  try:
    req = Request(('https://www.metacritic.com/browse/games/release-date/available/xbox-series-x/metascore?page=' + str(i)), headers = headers)
    response = urlopen(req)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
  except HTTPError as e:
    print(e.status, e.reason)
  except URLError as e:
    print(e.reason)

  soup_jogos = soup.findAll('td', {'class': 'clamp-summary-wrap'})

  for index, soup_jogo in enumerate(soup_jogos):
    game['Name'] = soup_jogo.find('a', {'class': 'title'}).find('h3').get_text()
    spans = soup_jogo.find('div', {'class': 'clamp-details'}).findAll('span')
    game['Release date'] = spans[-1].get_text()

    metascore = soup_jogo.find('div', {'class': 'clamp-metascore'})
    userscore = soup_jogo.find('div', {'class': 'clamp-userscore'})
    game['Metascore'] = metascore_score(metascore)
    game['Userscore'] = userscore_score(userscore)

    games.append(game.copy())

In [6]:
game

{'Name': 'CrossfireX',
 'Release date': 'February 10, 2022',
 'Metascore': '38',
 'Userscore': '3.3'}

In [7]:
len(games)

281

## Exportanto os dados como Dataframe

Gerando o Dataframe

In [8]:
dt_xseries_metacritic = pd.DataFrame(games)

In [9]:
dt_xseries_metacritic

,Name,Release date,Metascore,Userscore
0,Elden Ring,"February 25, 2022",96,7.7
1,Hades,"August 13, 2021",93,8.6
2,The Stanley Parable: Ultra Deluxe,"April 27, 2022",93,8.4
3,Forza Horizon 5,"November 5, 2021",92,8.2
4,Ori and the Will of the Wisps,"November 10, 2020",92,8.6
...,...,...,...,...
276,Music Racer: Ultimate,"March 4, 2022",50,tbd
277,Blackwind,"January 20, 2022",48,tbd
278,Balan Wonderworld,"March 26, 2021",47,7.2
279,Krut: The Mythic Wings,"July 12, 2022",43,tbd


In [11]:
dt_xseries_metacritic.drop_duplicates(subset = 'Name').shape

(281, 4)

In [12]:
dt_xseries_metacritic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          281 non-null    object
 1   Release date  281 non-null    object
 2   Metascore     281 non-null    object
 3   Userscore     281 non-null    object
dtypes: object(4)
memory usage: 8.9+ KB


Exportando o Dataframe

In [13]:
metacritic_xseries = dt_xseries_metacritic.to_csv('metacritic_xseries.csv', index = False, encoding = 'utf-8-sig')